In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv
import re

# 設置 Selenium 驅動
options = Options()
options.add_argument("--headless")  # 如果需要顯示瀏覽器，請去掉此行
service = Service("/opt/homebrew/bin/chromedriver")  # 指定 ChromeDriver 的路徑
driver = webdriver.Chrome(service=service, options=options)

# 打開Google Travel的航班頁面
url = "https://www.google.com/travel/flights/booking?tfs=CBwQAhplEgoyMDI0LTEwLTIzIh8KA1RQRRIKMjAyNC0xMC0yMxoDSEtHKgJDWDIDNDc5Ih8KA0hLRxIKMjAyNC0xMC0yMxoDU1lEKgJDWDIDMTAxagwIAhIIL20vMGZ0a3hyBwgBEgNTWURAAUgBcAGCAQsI____________AZgBAg&tfu=CnRDalJJZVZOME1tSkpVbGs1UkdkQlRHWmxZa0ZDUnkwdExTMHRMUzB0ZEd4aVpIVXlNa0ZCUVVGQlIyTkVORXRuUW1OdFFtOUJFZ3REV0RRM09YeERXREV3TVJvTENLckxBUkFBR2dOVVYwUTRISENXK0FRPRICCAUiAA&authuser=0"
driver.get(url)

driver.implicitly_wait(10)

# 抓取出發日期
departure_date_element = driver.find_element(By.XPATH, "//span[contains(@class, 'mv1WYe')]").get_attribute("innerHTML")[:9]
departure_date = departure_date_element.strip()

# 抓取出發時間
departure_time_element = driver.find_element(By.XPATH, "//div[@class='wtdjmc YMlIz ogfYpf tPgKwe']").get_attribute("aria-label")
departure_time = departure_time_element.split("：")[-1].strip()  # 抓取時間部分

# 抓取抵達時間
arrival_time_element = driver.find_element(By.XPATH, "//div[@class='XWcVob YMlIz ogfYpf tPgKwe']").get_attribute("aria-label")
arrival_time = arrival_time_element.split("：")[-1].strip()  # 抓取時間部分

# 獲取所有符合條件的元素
airport_elements = driver.find_elements(By.XPATH, "//span[contains(@class, 'qeoz6e HKHSfd')]/following-sibling::span[@dir='ltr']")
# 抓取出發和抵達機場代碼
departure_airport = airport_elements[0].get_attribute("innerHTML").strip("()")  # 第一個是出發機場
arrival_airport = airport_elements[1].get_attribute("innerHTML").strip("()")    # 第二個是抵達機場

# 抓取航空公司
airline = driver.find_element(By.XPATH, "//div[contains(@class, 'sSHqwe')]/span[1]").text

# 抓取行程時間
travel_time_element = driver.find_element(By.XPATH, "//div[@class='gvkrdb AdWm1c tPgKwe ogfYpf']").get_attribute("innerHTML")

# 移除 "Travel time: " 前缀（如果存在）
flight_duration = travel_time_element.replace("路程時間：", "").strip()

# 使用正則表達式提取
match = re.search(r'(\d+ 小時 \d+ 分鐘)', travel_time_element)
if match:
    flight_duration = match.group(1)
    
# 抓取停靠站數量
try:
    layover_element = driver.find_element(By.XPATH, "//div[@class='EfT7Ae AdWm1c tPgKwe']//span[@class='ogfYpf']").get_attribute("aria-label")
    layover = layover_element.split(" flight.")[0]  # 提取 "1 stop" 或 "Non-stop"
except NoSuchElementException:
    layover = "Non-stop"

if layover != "直達航班。":
    # 抓取停留時間
    layover_info_element = driver.find_element(By.XPATH, '//div[@class = "tvtJdb eoY5cb y52p7d"]').get_attribute("innerHTML")
    
    # 更新正則表達式，支持「小時和分鐘」或僅「分鐘」
    time_pattern = r'(\d+\s*小時\s*\d+\s*分鐘|\d+\s*分鐘)'
    match = re.search(time_pattern, layover_info_element)
    
    if match:
        layover_time = match.group(1)
    else:
        layover_time = "未找到停留時間"
else:
    layover_time = "Non-stop"


# 抓取機型
aircraft = driver.find_element(By.XPATH, '//div[@class="MX5RWe sSHqwe y52p7d"]/span[@class = "Xsgmwe"][last()]').get_attribute("innerHTML")

# 抓取航班代碼
flight_number_element = driver.find_element(By.XPATH, '//div[@class="MX5RWe sSHqwe y52p7d"]/span[contains(@class, "Xsgmwe")][2]').get_attribute("innerHTML")
flight_number = flight_number_element.replace('&nbsp;', ' ').strip()  # 去除前後空白

# 抓取艙等
cabin_class = driver.find_element(By.XPATH, '//span[contains(@class, "Xsgmwe")]/div').get_attribute("innerHTML")

# 輸出結果
print(f"出發日期: {departure_date}")
print(f"出發時間: {departure_time}")
print(f"出發機場代號: {departure_airport}")
print(f"抵達時間: {arrival_time}")
print(f"抵達機場代號: {arrival_airport}")
print(f"航空公司: {airline}")
print(f"停靠站數量: {layover}")
print(f"停留時間: {layover_time}")
print(f"飛行時間: {flight_duration}")
print(f"機型: {aircraft}")
print(f"航班代碼: {flight_number}")
print(f"艙等: {cabin_class}")

# 抓取價格歷史
elements = driver.find_elements(By.XPATH, "//*[name()='g' and @class='ke9kZe-LkdAo-RbRzK-JNdkSc pKrx3d']")
price_history = [element.get_attribute("aria-label") for element in elements]
for entry in price_history:
    print(entry)

# 匯出至 CSV，改為追加模式
csv_file = 'google_flights_data.csv'


# 檢查檔案是否已存在
file_exists = False
try:
    with open(csv_file, 'r', encoding='utf-8-sig'):
        file_exists = True
except FileNotFoundError:
    file_exists = False

# 將資料轉為 column 結構，每個屬性對應一列
flight_data = {
    "出發日期": departure_date,
    "出發時間": departure_time,
    "出發機場代號": departure_airport,
    "抵達時間": arrival_time,
    "抵達機場代號": arrival_airport,
    "航空公司": airline,
    "停靠站數量": layover,
    "停留時間": layover_time,
    "飛行時間": flight_duration,
    "機型": aircraft,
    "航班代碼": flight_number,
    "艙等": cabin_class,
    "價格歷史": "; ".join(price_history)  # 把價格歷史合併到一個欄位
}

# 打開或創建 CSV 文件，並以追加模式寫入
with open(csv_file, mode='a', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    
    if not file_exists:
        # 如果檔案不存在，寫入屬性標題
        writer.writerow(flight_data.keys())
    
    # 寫入航班的資料到新的一列
    writer.writerow(flight_data.values())
    
# 關閉瀏覽器
driver.quit()

出發日期: 10月23日 週三
出發時間: 晚上9:05。
出發機場代號: TPE
抵達時間: 上午11:50 (星期四, 10月 24)。
抵達機場代號: HKG
航空公司: 國泰航空
停靠站數量: 需轉機 1 次的航班。
停留時間: 50 分鐘
飛行時間: 11 小時 45 分鐘
機型: Airbus A321neo
航班代碼: CX 479
艙等: 經濟艙
60 天前 - $15,415
59 天前 - $15,401
58 天前 - $15,370
57 天前 - $15,370
56 天前 - $15,370
55 天前 - $15,363
54 天前 - $15,363
53 天前 - $15,357
52 天前 - $15,328
51 天前 - $15,350
50 天前 - $15,350
49 天前 - $15,350
48 天前 - $15,325
47 天前 - $15,288
46 天前 - $15,279
45 天前 - $15,278
44 天前 - $15,288
43 天前 - $15,288
42 天前 - $15,288
41 天前 - $15,269
40 天前 - $15,266
39 天前 - $15,281
38 天前 - $15,295
37 天前 - $15,285
36 天前 - $14,264
35 天前 - $14,264
34 天前 - $14,275
33 天前 - $14,285
32 天前 - $14,285
31 天前 - $14,290
30 天前 - $14,280
29 天前 - $14,280
28 天前 - $14,280
27 天前 - $14,269
26 天前 - $14,277
25 天前 - $14,288
24 天前 - $14,284
23 天前 - $14,288
22 天前 - $14,288
21 天前 - $14,288
20 天前 - $14,259
19 天前 - $14,247
18 天前 - $14,256
17 天前 - $14,271
16 天前 - $14,273
15 天前 - $14,273
14 天前 - $14,273
13 天前 - $14,279
12 天前 - $14,292
11 天前 - $14,278
10 天前 - $14,274
9